![Cisco Networking Academy](images/cisco.png)

# Lab: Power Plant Earthquake Emergency Shutdown System (Optional Lab)

## Lab Topology
![lab-topo](images/topology.png)

## Objectives
 * Set up an mLab user account for a cloud MongoDB database
 * Connect electronic circuits
 * Use PL-App with Two Raspberry Pis

## 	Background / Scenario 
This lab demonstrates the use of HTTP based REST API calls to integrate third party services into your application. You will learn how to store and retrieve JSON data in a MongoDB database running in the cloud, build applications that store the device state in a central place, and build multiple clients that can react based on the stored data.

MongoDB is an open source NoSQL document-oriented database that directly maps to applications. It provides a mechanism allowing the integration of data in certain types of applications to become much easier and faster. While it has native support for most programming languages, this lab will for simplicity use an HTTP based REST API access to an mLab hosted MongoDB instance (http://docs.mlab.com/data-api/).

The application simulates a simplified factory emergency shutdown system where a single control board can be used by an operator to update the state of an alarm variable in a central database. Multiple emergency client boards periodically check for the alarm property value and trigger an alarm on an external electrical system. For simplicity in this lab you will only be turning on or off LEDs. In real world practice, the control boards could turn off gas pumps, close fire doors, or perform other vital tasks.

*Instructor Note: Because this lab uses two RaPi boards, it is recommended that the instructor organize the class in pairs. Student pairs will elect a board to act as control board while the other as emergency board.*


## Required Resources 
 * Prototyping Lab with two Raspberry Pi boards that are configured and imaged with PL-App
 * Internet connection with no traffic filtering for port 443 (https)
 * mLab account

# Task 1: Set up an mLab user account for a cloud MongoDB database

## Step 1:	Register a free user account at mLab
If you currently do not have an mLab user account, go to https://mlab.com/signup/ to get one for free.

## Step 2:	Setup a new MongoDB database using the mLab cloud
**a.**	Sign In and click **Create new** from the navigation menu to create a new **MongoDB** database.
![mongo01](images/mongo01.png)

**b.**	Select any Cloud provider from the list where your MongoDB instance is going to be created. 

**c.**	Select a **Sandbox** offering with a free, up to 0.5GB size, database. Click **Continue**.
![mongo02](images/mongo02.png)

**d.**	Select an AWS Region and click **Continue**.
![mongo03](images/mongo03.png)

**e.**	Select a Database name which must be unique (you can use a combination of your username + some additional letters) and click **Continue**
![mongo04](images/mongo04.png)

**f.**	Review the chosen settings and click **Submit Order**.
![mongo05](images/mongo05.png)

**g.**	Once a database has been created, enter it by clicking the database name.
![mongo06](images/mongo06.png)

**h.**	Inside of the database, create a new collection of JSON based documents.
![mongo07](images/mongo07.png)
![mongo08](images/mongo08.png)

**i.**	Enter the newly created collection of JSON based documents:
![mongo09](images/mongo09.png)

**j.**	Inside the collection, create a new JSON based document.
![mongo10](images/mongo10.jpg)

**k.**	The new JSON document will represent the state of the emergency alarm. 
The **control board** will update this document with the desired state of the alarm, while the **emergency board** will retrieve this document to check the current state of the alarm parameter. The alarm parameter will be a Boolean variable, having true or false state.
![mongo11](images/mongo11.png)

`{“alarm”:false}`

**Note**: *Once the document is created, you should see it on the previous page among your Collection.*
![mongo12](images/mongo12.png)
 

## Step 3:	Getting your mLab’s REST API Secret Key
**a.**	To enable HTTP RESP API access, access the user account page by clicking your username in the top right menu of the page.
![mongo13](images/mongo13.jpg)

**b.**	At the bottom of your user profile page, click **Enable Data API Access** to enable HTTP based REST API access to your mLab’s databases:
![mongo14](images/mongo14.jpg)

**Note**: *Remember your **API key** from the user account page. You will need it for your application that is going to be running on the Raspberry Pi boards. You will need the mLab **API key** every time you want to update or retrieve the documents stored in your database by using an HTTP or HTTPS GET/POST URL:*

`https://api.mlab.com/api/1/databases/{DB}/collections/{CO}/?apiKey=y2***HVXs`

*where `{DB}` is the database name you created, `{CO}` is the collection name you created, all followed at the end of the URL with your API Key.*

# Task 2: Connect electronic circuits

## Step 1:	Connect external LEDs using a breadboard to the Raspberry Pi Emergency Board
**a.**	Using red and green color LEDs, resistors, jumper cables and a breadboard connect the Raspberry Pi that is to be used as the **Emergency Board**.

**b.**	The logical schematic is outlined in the figure below. The anode leg of the red LED1 is to be connected to GPIO pin number 21 based on the BCM GPIO scheme (or pin number 40 based on the physical pin numbering scheme) of the Raspberry Pi version version 3, while the anode leg of the green LED2 is to be connected to the GPIO pin number 20 based on the BCM GPIO scheme (or pin number 38 based on the physical pin numbering scheme) of the Raspberry Pi version 3) .
![rpi01](images/rpi01.jpg)

**c.**	Connect the LEDs, two 330Ω resistors and jumper cables as shown on the physical diagram below to a breadboard and to the first Raspberry Pi. The physical connections are outlined in the figure below.
![rpi02](images/rpi02.jpg)

## Step 2:	Connect a button using a breadboard to the Raspberry Pi Control Board
**a.**	Using a push button, resistor, jumper cables and a breadboard connect the Raspberry Pi that is to be used as the **Control Board**.

**Note**: *The logical schematic is outlined in the figure below. The push button is connected to the input GPIO pin of the Raspberry Pi with a pull down resistor. This ensures that when the button is off, the GPIO pin is sensing a logical LOW state. Once the button is pressed and goes on, the current flows from the +3.3V pin directly to the input GPIO pin, thus sensing a logical HIGH state. The pull down leg of the push button S1 is to be connected to GPIO pin number 21 based on the BCM GPIO scheme (or pin number 40 based on the physical pin numbering scheme) of the Raspberry Pi version 3).*
![rpi03](images/rpi03.jpg)

**b.**	Connect the push button, a 10kΩ resistor and jumper cables as shown on the physical diagram below to a breadboard and to the second Raspberry Pi. The physical connections are outlined in the figure below.
![rpi04](images/rpi04.jpg)

# Task 3: Software to connect to dots

 ## Step 1: Import the needed Python modules

The `RPi.GPIO` Raspberry Pi Python's module is used to interact with the physical GPIO pins of the Raspberry Pi. The `RPi.GPIO` module provides functions to set various PIN numbering schemes, input or output mode of the GPIO pins and functions to either read the current state of an input pin, or set the state of an output pin.

The Python's `requests` module provides an easy to use HTTP library to send and receive messages to web servers. Since  RESTful API services, such as Cisco Spark APIs, are implemented as small web servers, this is an essential module.

Most of the messages that are exchanged between the API client and API server have a special formating. Most common are XML and JSON. Cisco Spark API uses JSON to encode messages. To work with JSON encoded data, in Python import the `json` module.

In [ ]:
# Import the GPIO modules to control the GPIO pins of the Raspberry Pi
import RPi.GPIO as GPIO
# Import the requests module to execute RESTful API calls with HTTP requests
import requests
# Import the json module to work with JSON encoded objects
import json
# Import the time module to control the timing of your application (e.g. add delay, etc.)
import time

## Step 2: Access the mLab’s RESTful API Endpoints

**a**. Set the following 3 variables:
 * mongoLabsDatabase
 * mongoLabsCollection
 * mongoLabsApiKey

with your settings from mlab.com and then execute the cell to set the `mongoLabsApiURI` variable.

The `mongoLabsApiURI` variable holds the RESTful API endpoint URL for your mlab database. Any time you want to read or write data to your mlab.com database, the RESTful API HTTP requests must use this URL.

In [ ]:
# Copy the MongoDB database name from here https://mlab.com:
mongoLabsDatabase = "!!!-REPLACE-WITH-YOUR-MLAB.COM-DB-NAME-!!!!"
# the collection name inside of your database:
mongoLabsCollection = "!!!-REPLACE-WITH-YOUR-MLAB.COM-COLLECTION-NAME-!!!!"
# and the API KEY from your https://mlab.com/user (the API user must have Enabled Data API Access)
mongoLabsApiKey = "!!!-REPLACE-WITH-YOUR-MLAB.COM-DB-API-KEY!!!!"

# The RESP API URL Endpoint for the specified collection in your database:
mongoLabsApiURI = "https://api.mlab.com/api/1/databases/" + mongoLabsDatabase + "/collections/" + mongoLabsCollection + "/?apiKey=" + mongoLabsApiKey

**b**. Set a new record to your mlab.com database. To add a new record to the database, the RESTful API HTTP call must use the PUT HTTP method and the record data must be json encoded.

With the `requests` module, it's easy to create a new PUT HTTP request with the `requests.put()` function.

In [ ]:
# a value to store in the database - can be either True or False
alarmValue = True

# requests.put - PUT a new "alarm" record in the DB
r = requests.put( mongoLabsApiURI,
                  json = {"alarm": alarmValue}   # JSON encoded data
)

# verify if the response indicates a successfull requests (http code 200)
if (r.status_code != 200):                  # if response from request is not 
    print("Something wrong has happened:") #200/OK, then alert!
    print("ERROR CODE: " + str(r.status_code))
    print("ERROR RESPONSE: " + r.text)
else:
    print("All OK, record stored.")

**c**. Retrieve the records from your mlab.com database. To retrieve records from the database, the RESTful API HTTP call must use the GET HTTP method. The returned data records are json encoded in the reply.

With the `requests` module, it's easy to create a new GET HTTP request with the `requests.get()` function.

In [ ]:
# requests.get - GET method to get the records data from the database
r = requests.get(mongoLabsApiURI)

if (r.status_code != 200):
    print("Something wrong has happened:")
    print("ERROR CODE: " + str(r.status_code))
    print("ERROR RESPONSE: " + r.text)
else:
    # the reply in the "r" object should be json encoed data
    jsonData = r.json()
    
    # to see a raw output of the json reply, uncomment the following line:
    #print(json.dumps(jsonData, indent=4))
    
    # the actuall data should be an array of records
    # the first item in the array is the most recent record in the database:
    firstRecord = jsonData[0]

    # each record should have the "alarm" field:
    isAlarm = firstRecord["alarm"]
    
    print("State of the last alarm record in the database: " + str(isAlarm))

**d**. Try to modify the `alarmValue` in the previous second cell from `True` to `False` (or oposite) and execute the cells to see the new `alarmValue` is successfully stored as well as retrieved from the dabase using the RESTful API calls of the previous two code cells.

## Step 3 on the emergency board: Checking for alarms

**Note**: *Only run the code cells in this step on the Raspberry Pi board that functions as the **emergency board**.*

**a**. Set the GPIO pin numbering scheme to BCM.

**b**. Set the pins with the LEDs to the OUTput mode.

In [ ]:
# Set the desired pin numbering scheme:
GPIO.setmode(GPIO.BCM)

# GPIO PINs where the LEDs are connected 
#  pin numbering based on the BCM scheme
# the PIN of the green LED - OK
GreenLEDPin = 20
# the PIN of the red LED - ALARM
RedLEDPin   = 21

# Setup the direction of the GPIO pins - either INput or OUTput 
#  In case of the PINs that connect LEDs, they must be set to OUTput mode:
GPIO.setup(GreenLEDPin, GPIO.OUT)
GPIO.setup(RedLEDPin, GPIO.OUT)

**c**. Execute the cell below to verify the LEDs are blinking and the GPIO access is working.

In [ ]:
for i in range(2):
    print("ON")
    GPIO.output(GreenLEDPin, True) # True = set 3.3V on the pin
    GPIO.output(RedLEDPin,   True) # True = set 3.3V on the pin
    time.sleep(1)
    print("OFF")
    GPIO.output(GreenLEDPin, False) # False = set 0V on the pin
    GPIO.output(RedLEDPin,   False) # False = set 0V on the pin
    time.sleep(1)

**d**. Execute the cell below to start the Emergency Shutdown system on the emergency board.
 1. The code will be running in a while loop until manually stopped or an error occurs. 
 2. In each itteration of the loop, a new API request is made to retrieve the record from the mlab.com database. 
 3. Based on the returned records, specifically the "alarm" state of the most recent record, the LEDs connected to the GPIO pins of the Raspberry Pi will turn on or off based the the alarm state.
 4. Simultaneously with this, on the 2nd Raspberry Pi that is used as the control board, change the alarm state by pressing the button.

In [ ]:
print("Started the factory Emergency Shutdown system ...")

# loop forever
while True:
    # get the records from the mlab.com database:
    r = requests.get(mongoLabsApiURI)
    if (r.status_code != 200):
        print("Something wrong has happened:")
        print("ERROR CODE: " + str(r.status_code))
        print("ERROR RESPONSE: " + r.text)
        assert()  # stop the application at this point
    # get the most recent alarm status from the recods
    isAlarm = r.json()[0]["alarm"]
    if(isAlarm):    # if the alarm is on (True)
        print("RUN RUN RUN")
        GPIO.output(GreenLEDPin, False)
        GPIO.output(RedLEDPin,   True)
    else:
        print("OK")
        GPIO.output(GreenLEDPin, True)
        GPIO.output(RedLEDPin,   False)
    # wait one second before the next itteration
    # this is important to not to overload the online service with lot of API requests
    time.sleep(1)

## Step 3 on the control board: Controlling alarm states

**Note**: *Only run the code cells in this step on the Raspberry Pi board that functions as the **control board**.*

**a**. Set the GPIO pin numbering scheme to BCM.

**b**. Set the pin with the button to the INput mode.

In [ ]:
# Set the desired pin numbering scheme:
GPIO.setmode(GPIO.BCM)

# GPIO PIN where the button is connected 
#  pin numbering based on the BCM scheme
# the PIN of the button - the RED PANIC EMERGENCY button ;-)
buttonPin = 21

# Setup the direction of the GPIO pins - either INput or OUTput 
#  In case of the PIN that connects the button, it must be set to INput mode:
GPIO.setup(buttonPin, GPIO.IN)

**c**. Execute the cell bellow to verify the LEDs are blinking and the GPIO access is working. Press and release the button to see the changes.

In [ ]:
# first read the button state 
#  and store it in the variables
buttonState = previousItterationButtonState = GPIO.input(buttonPin)

print("Button state is: " + str(buttonState))
print("Try to press the button...")

# loop forever - stop this cell by clicking on the STOP button on the left part of the cell
while True:
    # read the state of the pin where the button is connected
    #  returns True is the button is pressed and the possitive voltage is detected on the pin
    buttonState = GPIO.input(buttonPin)
    # compare the new state with the previous state 
    if(buttonState != previousItterationButtonState):
        print("Button change. New state is: " + str(buttonState))
    # save the state of this itteration
    previousItterationButtonState = buttonState

**d**. Execute the cell bellow to start the Emergency Shutdown system on the control board. (Make sure the above code in Step 3c is not actively running before executing the code below.)
 1. The code will be running in a while loop until manually stoped or an error occurs. 
 2. In each itteration of the loop, the state of the button is checked. 
 3. If change is detected, the new button state representing the alarm state is put using the API request as a new recod to the mlab.com database.
 4. Simultaneously with this, the other Raspberry Pi board that was setup as the emergecy board can check for the new alarm records on the mlab.com database and accordingly set the LEDs.

In [ ]:
print("Started the factory Control system ...")
# first read the button state 
#  and store it in the variables
buttonState = previousItterationButtonState = GPIO.input(buttonPin)

# loop forever - stop this cell by clicking on the STOP button on the left part of the cell
while True:
    # read the state of the pin where the button is connected
    #  returns True if the button is pressed and positive voltage is detected on the pin
    buttonState = GPIO.input(buttonPin)
    # compare the new state with the previous state 
    if(buttonState != previousItterationButtonState):
        # convert the buttonState returned by the GPIO.input() from a number to a boolean 
        #  (1=True, 0=False)
        alarmValue = True if buttonState == 1 else False
        print("Button change. New state is: " + str(buttonState) + " and alarm is: " + str(alarmValue))
        print("Updating alarm value in DB using the RESP API call over HTTP.")
        r = requests.put(mongoLabsApiURI,        # PUT new "alarm" in the DB
                         json = {"alarm": alarmValue}   # update JSON data
                         )
        if (r.status_code != 200):	        # if response from request is not 
            print("Something wrong has happened:") #200/OK, then alert!
            print("ERROR CODE: " + str(r.status_code))
            print("ERROR RESPONSE: " + r.text)
            assert()  # stop the application at this point
    # save the state of this itteration
    previousItterationButtonState = buttonState

## Reflection

What are the geographical limitations regarding the placement of the control board and the emergency board?

###### &copy; 2017 Cisco and/or its affiliates. All rights reserved. This document is Cisco Public.